In [1]:
import findspark
findspark.init()

In [2]:
import json

In [3]:
# from pyspark_ai import SparkAI

# spark_ai = SparkAI()
# spark_ai.activate()  # active partial functions for Spark DataFrame

In [4]:
import pyspark
from pyspark.sql import functions as F

In [5]:
import logging

logger = logging.getLogger(__name__)

In [6]:
from code_master import CodeMaster

In [7]:
from pyspark.sql import SparkSession, DataFrame
spark = SparkSession.builder.appName('abc').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/17 13:53:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
xyz = spark.createDataFrame(
    [
        ("Normal", "Cellphone", 6000),
        ("Normal", "Tablet", 1500),
        ("Mini", "Tablet", 5500),
        ("Mini", "Cellphone", 5000),
        ("Foldable", "Cellphone", 6500),
        ("Foldable", "Tablet", 2500),
        ("Pro", "Cellphone", 3000),
        ("Pro", "Tablet", 4000),
        ("Pro", "Tablet", 4000),
        ("Pro Max", "Cellphone", 4500)
    ],
    ["a", "c", "v"]
)

In [9]:
xyz.show(20)

+--------+---------+----+
|       a|        c|   v|
+--------+---------+----+
|  Normal|Cellphone|6000|
|  Normal|   Tablet|1500|
|    Mini|   Tablet|5500|
|    Mini|Cellphone|5000|
|Foldable|Cellphone|6500|
|Foldable|   Tablet|2500|
|     Pro|Cellphone|3000|
|     Pro|   Tablet|4000|
|     Pro|   Tablet|4000|
| Pro Max|Cellphone|4500|
+--------+---------+----+



In [10]:
# auto sales data from https://www.carpro.com/blog/full-year-2022-national-auto-sales-by-brand
data = [('Toyota', 1849751, -9), ('Ford', 1767439, -2), ('Chevrolet', 1502389, 6),
        ('Honda', 881201, -33), ('Hyundai', 724265, -2), ('Kia', 693549, -1),
        ('Jeep', 684612, -12), ('Nissan', 682731, -25), ('Subaru', 556581, -5),
        ('Ram Trucks', 545194, -16), ('GMC', 517649, 7), ('Mercedes-Benz', 350949, 7),
        ('BMW', 332388, -1), ('Volkswagen', 301069, -20), ('Mazda', 294908, -11),
        ('Lexus', 258704, -15), ('Dodge', 190793, -12), ('Audi', 186875, -5),
        ('Cadillac', 134726, 14), ('Chrysler', 112713, -2), ('Buick', 103519, -42),
        ('Acura', 102306, -35), ('Volvo', 102038, -16), ('Mitsubishi', 102037, -16),
        ('Lincoln', 83486, -4), ('Porsche', 70065, 0), ('Genesis', 56410, 14),
        ('INFINITI', 46619, -20), ('MINI', 29504, -1), ('Alfa Romeo', 12845, -30),
        ('Maserati', 6413, -10), ('Bentley', 3975, 0), ('Lamborghini', 3134, 3),
        ('Fiat', 915, -61), ('McLaren', 840, -35), ('Rolls-Royce', 460, 7)]

dq = spark.createDataFrame(data, ["Brand", "US_Sales_2022", "Sales_Change_Percentage"])

In [11]:
dq.show(20)

+-------------+-------------+-----------------------+
|        Brand|US_Sales_2022|Sales_Change_Percentage|
+-------------+-------------+-----------------------+
|       Toyota|      1849751|                     -9|
|         Ford|      1767439|                     -2|
|    Chevrolet|      1502389|                      6|
|        Honda|       881201|                    -33|
|      Hyundai|       724265|                     -2|
|          Kia|       693549|                     -1|
|         Jeep|       684612|                    -12|
|       Nissan|       682731|                    -25|
|       Subaru|       556581|                     -5|
|   Ram Trucks|       545194|                    -16|
|          GMC|       517649|                      7|
|Mercedes-Benz|       350949|                      7|
|          BMW|       332388|                     -1|
|   Volkswagen|       301069|                    -20|
|        Mazda|       294908|                    -11|
|        Lexus|       258704

In [12]:
data = {}

In [13]:
data['xyz'] = xyz
data['dq'] = dq

In [14]:
data

{'xyz': DataFrame[a: string, c: string, v: bigint],
 'dq': DataFrame[Brand: string, US_Sales_2022: bigint, Sales_Change_Percentage: bigint]}

In [15]:
f = open ('sel_test.json', "r")
pipeline_dict = json.loads(f.read())

In [16]:
pipeline_dict

{'module_name': 'ar',
 'version': '0.1',
 'pipeline': {'stage_1': {'select': {'table_name': 'xyz',
    'cols_to_select': ['a', 'v'],
    'drop': ['p'],
    'dedup': 'True',
    'output': 'xyz'}},
  'stage_2': {'select': {'table_name': 'dq', 'output': 'dq'}}}}

In [17]:
#code master

code_master_obj = CodeMaster(pipeline=pipeline_dict, data=data, spark=spark)

In [18]:
df, df_dict = code_master_obj.build_pipeline()

In [19]:
df_dict

{'xyz': DataFrame[a: string, v: bigint],
 'dq': DataFrame[Brand: string, US_Sales_2022: bigint, Sales_Change_Percentage: bigint]}

In [21]:
df_dict['xyz'].show(20)

+--------+----+
|       a|   v|
+--------+----+
|  Normal|6000|
|  Normal|1500|
|    Mini|5500|
|Foldable|6500|
|    Mini|5000|
|Foldable|2500|
|     Pro|3000|
|     Pro|4000|
| Pro Max|4500|
+--------+----+



In [16]:
pipeline = pipeline_dict['pipeline']

In [20]:
for stage in pipeline.keys():
    print (stage)
    print(pipeline.get(stage))
    stage_operations = pipeline.get(stage)
    for oper in stage_operations.keys():
        print(oper)
        print(stage_operations.get(oper))

stage_1
{'select': {'table_name': 'xyz', 'cols_to_select': ['a', 'c', 'v', 'p'], 'drop': ['p'], 'dedup': 'True'}}
select
{'table_name': 'xyz', 'cols_to_select': ['a', 'c', 'v', 'p'], 'drop': ['p'], 'dedup': 'True'}
stage_2
{'select': {'table_name': 'dq'}}
select
{'table_name': 'dq'}


In [64]:
sel_details = pipeline['pipeline']['stage_1']['select']
sel_details

{'table_name': 'xyz',
 'cols_to_select': ['a', 'c', 'v', 'p'],
 'drop': ['p'],
 'dedup': 'True'}

In [65]:
df1 = selection(sel_details)

In [1]:
df1.show(20)

NameError: name 'df1' is not defined

In [63]:
def selection(details:dict):
    try:
        df_sel = data[details["table_name"]]
    except:
        logger.error("No dataframe mentioned to select.")
        exit()

    if 'cols_to_select' in details.keys():
        try:
            df_sel = df_sel.select(*details["cols_to_select"])
        except:
            logger.info('Column selection %s for table %s is incorrect.', str(details["cols_to_select"]), details["table_name"])
            exit()
    
    if 'drop' in details.keys():
        df_sel = df_sel.drop(*details["drop"])

    if 'dedup' in details.keys():
        if details['dedup'].upper() == "TRUE":
            df_sel = df_sel.drop_duplicates()

    return df_sel